In [1]:
import pymongo
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import time
from datetime import date
from datetime import timedelta
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import json
import requests
from pymongo.errors import BulkWriteError
from pprint import pprint
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import emoji
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from itertools import chain
from collections import Counter
import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump, load
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
time = 72
date_of_testing ='12/28/23'
date_of_testing_datetime = datetime.strptime(date_of_testing, '%m/%d/%y')
time = date_of_testing_datetime - timedelta(hours=time)

In [3]:
time

datetime.datetime(2023, 12, 25, 0, 0)

In [4]:
date_of_testing_datetime

datetime.datetime(2023, 12, 28, 0, 0)

In [6]:
## DATA FOR PREDICTION (it is yesterday data of the current data)
stock_name = 'TSLA'
header = {'Accept' : 'application/json',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language' :  'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6',
    'Connection': 'keep-alive',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Sec-Fetch-Dest':'empty'}


              # this name will come from flask frontend
main_data = []                    #  All data will store here
api_next_list = []


time = 24                             # Change according to number of days
current_time = datetime.now()                 # It show current time
time = current_time - timedelta(hours=time)   # it will go 24 hours back and give you a specific time and time
# time = 24
# date_of_testing ='12/15/23'
# date_of_testing_datetime = datetime.strptime(date_of_testing, '%m/%d/%y')
# time = date_of_testing_datetime - timedelta(hours=time)

#Initializing the request and converts it into json
url = f'https://api.stocktwits.com/api/2/streams/symbol/{stock_name}.json?filter=top&limit=22'  
request_success = requests.get(url,headers=header)   
json_data_dict = request_success.json()


#Taking all the usefull data from the json data
usefull_data = (json_data_dict['messages']) 

#picking the next api address for further scrapping
next_api_first = json_data_dict['cursor']['max']
next_api_first = str(next_api_first)
next_api_first = '&max='+next_api_first



# Retrieving the last date from twit to break the first loop
last_twit_date_1 = (json_data_dict['messages'][-1])
last_twit_date_1 = last_twit_date_1['created_at']
last_twit_date_1 = re.sub(pattern = "[T-Z]+",repl = ' ',string=last_twit_date_1) # eliminating alphabets
last_twit_date_1 = datetime.strptime(last_twit_date_1, "%Y-%m-%d  %H:%M:%S ")  # changing datatype



#Looping in the usefull_data
for data in usefull_data:
    # Getting the twit date from  usefull_data and cleaning is done
    twit_date = data['created_at']
    twit_date = re.sub(pattern = "[T-Z]+",repl = ' ',string=twit_date)
    twit_date = datetime.strptime(twit_date, "%Y-%m-%d  %H:%M:%S ")   

    if twit_date > time:
        twit = data['body']
        signal = data['entities']['sentiment']
        if signal == None:
            signal = 'Null'
        else :
            signal = signal['basic']
        stock_twits_data = {'stock_name':stock_name ,'Date':twit_date,'Twits': twit,'Signal': signal,'Cleaning_status':'pending','Cleaning_status_lemmatization':'pending'}
        main_data.append(stock_twits_data)
    if twit_date < time:
        if len(main_data) == 0:
            print('No twits found')
        else:
            print('twits are there')
        break
    try:
        if time < last_twit_date_1.strptime('%H:%M:%S'):
            break
    except:
        pass

#########################################################################################################################################    
while (True):
    loop_break = []
    url =  f'https://api.stocktwits.com/api/2/streams/symbol/{stock_name}.json?filter=top&limit=22{next_api_first}'
    request_success = requests.get(url,headers=header)   
    json_data_dict = request_success.json() 
    next_api_first = json_data_dict['cursor']['max']
    next_api_first = str(next_api_first)
    next_api_first = '&max='+next_api_first
    usefull_data = (json_data_dict['messages'])
    for data in usefull_data:
        twit_date = data['created_at']
        twit_date = re.sub(pattern = "[T-Z]+",repl = ' ',string=twit_date)
        twit_date = datetime.strptime(twit_date, "%Y-%m-%d  %H:%M:%S ")   
        if twit_date > time:
            twit = data['body']
            signal = data['entities']['sentiment']
            if signal == None:
                signal = 'Null'
            else :
                signal = signal['basic']
            stock_twits_data = {'stock_name':stock_name ,'Date':twit_date,'Twits': twit,'Signal':signal,'Cleaning_status':'pending','Cleaning_status_lemmatization':'pending'}
            main_data.append(stock_twits_data)



    if twit_date < time:
        print('while loop break')
        break
print(main_data)
if not main_data:    
    print('No data is found')
else :
    df = pd.DataFrame(main_data)
    pd.set_option('display.max_colwidth', None)


    ##Removing Links from dataframe
    print('---------------------------------Links------------------------------------')
    df['Twits'] = df['Twits'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


    ## Lowercase the whole twits column
    print('---------------------------------lowercase------------------------------------')
    df['Twits'] = df['Twits'].apply(lambda x: x.lower())


    ## Eliminating the pumctuation 
    print('----------------------------eliminate punctuation-------------------------------')
    df['Twits'] = df['Twits'].str.replace('[^\w\s]',' ',regex=True)


    ##Eliminating stopwords 
    def remove_stopwords(x):
        return " ".join([word for word in str(x).split() if word not in stop_words])
    stop_words = set(stopwords.words('english'))
    print('----------------------------removing stopwords-------------------------------')
    df['Twits'] = df['Twits'].apply(lambda x : remove_stopwords(x))


    ##Eliminating Emoji from dataframe
    print('----------------------------removing emojis-------------------------------')
    df['Twits'] = df['Twits'].apply(lambda s: emoji.replace_emoji(s, ''))

    # Stemming on dataframe
    print('----------------------------stemming is done-------------------------------')
    stemmer = PorterStemmer()
    df['Twits'] = df['Twits'].str.split()
    df['Twits'] = df['Twits'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
    df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))

    # Removing number str from column twits
    print('----------------------------removing number is done-------------------------------')
    df['Twits'] = df['Twits'].replace('\d+', '', regex=True)

    # Removing the words which contain only 1 letter
    print('----------------------------removing 1 letter word is done-------------------------------')
    df['Twits'] = df['Twits'].str.split()
    for text in df['Twits']:
        for word in text:
            if len(word) == 1:
                text.remove(word)
    df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))
    df_yesterday = df.drop(['stock_name','Date','Cleaning_status','Cleaning_status_lemmatization','Signal'], axis=1)
    df_yesterday = df_yesterday.drop_duplicates()
    print(df_yesterday)

while loop break
[{'stock_name': 'TSLA', 'Date': datetime.datetime(2024, 2, 19, 10, 4, 31), 'Twits': '$TSLA chart of the day https://tradernomad.beehiiv.com/p/chart-day-c32c', 'Signal': 'Null', 'Cleaning_status': 'pending', 'Cleaning_status_lemmatization': 'pending'}, {'stock_name': 'TSLA', 'Date': datetime.datetime(2024, 2, 19, 9, 59, 16), 'Twits': '$TSLA  it is down by 2 EUR in German stock exchange', 'Signal': 'Null', 'Cleaning_status': 'pending', 'Cleaning_status_lemmatization': 'pending'}, {'stock_name': 'TSLA', 'Date': datetime.datetime(2024, 2, 19, 9, 37, 9), 'Twits': '$TSLA $META pre-market updates...! http://make-money-bull-trades.stocktoinvest.wiki', 'Signal': 'Null', 'Cleaning_status': 'pending', 'Cleaning_status_lemmatization': 'pending'}, {'stock_name': 'TSLA', 'Date': datetime.datetime(2024, 2, 19, 9, 36, 57), 'Twits': '$SOFI $TSLA Analyst price target for next week...;&#39; http://smart-swing-strategy-chat.stocktoinvest.wiki', 'Signal': 'Bullish', 'Cleaning_status': 'pen

In [8]:
df_yesterday

,Twits
0,tsla chart day
1,tsla eur german stock exchang
2,tsla meta pre market updat
3,sofi tsla analyst price target next week
4,soun tsla analyst price target next week
...,...
251,tsla bought ytsl canada get paid almost divi also get lot upsid price tsla love
252,tsla last post happi weekend
253,tsla ran much high everi littl bit good news price everi littl bit bad news given fweedum guy propens tweet bad news overhyp good news steer clear shortcut wealth
255,tsla brim upper line break lower fanci chart lmao


In [9]:
df_yesterday_combine         = df_yesterday                                            # duplicating the df_yesterday
twits_prediction_lists       = df_yesterday_combine['Twits'].tolist()                  # Converting it into list
twits_prediction_lists_merge = " ".join(twits_prediction_lists)                        # joining each element of a list into a single element
dict_yesterday_twits_merge   = {'Twits':twits_prediction_lists_merge}                  # Initializing of creating dataframe  
df_yesterday_twits_merge     = pd.DataFrame(dict_yesterday_twits_merge,index=['Index'])# making dataframe
df_yesterday_twits_merge_unicode  = df_yesterday_twits_merge['Twits'].astype('unicode')# Converting it into unicode
loaded_tfidf_vectorizer = load('tfidf_vectorizer.pkl')
x_test_prediction_yesterday_merge = loaded_tfidf_vectorizer.transform(df_yesterday_twits_merge_unicode) 
print("Shape of yesterday_dataset:", x_test_prediction_yesterday_merge.shape)

Shape of yesterday_dataset: (1, 93352)


In [23]:
loaded_model = load('linear_regression_final.joblib')
predictions = loaded_model.predict(x_test_prediction_yesterday_merge)
print(f'The predicted value is {predictions}')
if predictions > 0.5 :
    print('Bullish')
else:
    print('Bearish')

The predicted value is [0.93380453]
Bullish


In [19]:
print(len(df_yesterday.index))
# len(df.index)

216
